# SENet

## Introduction

Convolution -> Feature recalibration

U : squeeze operation

Excitation operation : sample-specific activation


![Imgur](https://i.imgur.com/9UFjxDA.png)


SENet 장점 : 기존 네트워크에 바로 적용 가능



## Related work

Deep architectures : VGGNet, BN, ResNet, Highway network, Grouped convolutions, Multi-branch convolution etc.

Attention and gating mechanisms

Attention : input signal에서 가장 효율적인 부분에 자원 분배 가능하게 해줌 - localization 등에 응용


## Squeeze and Excitation Blocks

$${ { F }_{ tr }:X\rightarrow U,X\in { R }^{ { H }^{ ' }\times { W }^{ ' }\times { C }^{ ' } },U\in { R }^{ { H }\times { W }\times { C } } }$$

${ F }_{ tr }$ : Convoluation 연산

$V=\left[ { v }_{ 1 },{ v }_{ 2 },...,{ v }_{ c } \right] $ : 필터 집합 $V$와 각 필터의 파라미터. ${ v }_{ c }$는 c번째 필터의 파라미터

$U=\left[ { u }_{ 1 },{ u }_{ 2 },...,{ u }_{ C } \right]$ : $ { F }_{ tr }$을 통과한 결과물

$${ u }_{ c }={ u }_{ c }\ast X=\sum _{ s=1 }^{ { C }^{ ' } }{ { v }_{ c }^{ s }\ast { X }^{ s } } $$

${ v }_{ c }=\left[ { v }_{ c }^{ 1 },{ v }_{ c }^{ 2 },...,{ v }_{ c }^{ C' } \right] $ : ${ v }_{ c }^{ s }$ is 2D spatial kernel

${ X }=\left[ { x }^{ s },{ x }^{ 2 },...,x^{ C' } \right] $


### Squeeze: Global Information Embedding

네트워크 시작 부분일수록 receptive field가 작기 때문에 너무 국소적인 부분만 보는 현상이 생김. 이런 문제를 해결하기 위한 방법이 squeeze.

squeeze는 global spatial information을 channel descriptor로 압축시키는 것. 이를 위하여 global average pooling을 실시함.

$${ z }_{ c }={ F }_{ sq }\left( { u }_{ c } \right) =\frac { 1 }{ H\times W } \sum _{ i=1 }^{ H }{ \sum _{ j=1 }^{ W }{ { u }_{ c }(i,j) }  } $$



### Excitation: Adaptive Recalibration

Excitation은 squeeze operation을 통과해서 압축된 정보들을 사용하기 위한 작업. 목적은 channel-wise d ependency를 완전히 잡아내는 것.

- Flexible (채널 간 비선형적인 상호작용을 학습해야함)
- Must learn non-mutually-exclusive relationship

이를 위해서 sigmoid activation을 사용.

$$s={ F }_{ ex }\left( z,W \right) =\sigma ({ W }_{ 2 }\delta ({ W }_{ 1 }z))$$

위 식에서 델타는 ReLU.
모델 복잡성과 aid generalization을 위하여 FC layer 2개를 활성함수 옆에 쌓아서 bottleneck(ResNet에서 나온 그 보틀넥)을 만들었음. 여기서 ${W}_{1}$은 reduction ratio $\gamma $를 사용하여 차원수를 줄임. ${W}_{2}$를 통과하면 차원수가 늘어나게 됨.

최종 아웃풋은 아래와 같음

$${ \tilde { x }  }_{ c }={ F }_{ scale }({ u }_{ c },{ s }_{ c })=s_{ c }\cdot { u }_{ c }$$


${ \tilde { X }  }=\left[ { \tilde { x }  }_{ 1 },{ \tilde { x }  }_{ 2 },...,{ \tilde { x }  }_{ C } \right] $

${ F }_{ scale }|(u_{ c },{ s }_{ c })$ : Channel-wise multiplication between feature map and scalar


### Exemplars: SE-Inception and SE-ResNet

SE block의 장점 중 하나는 felxity로 어떤 convolution에도 쉽게 적용할 수 있다는 것. VGG나 AlexNet에도 바로 적용이 가능함.

![Imgur](https://i.imgur.com/NyGsIbX.png)

![Imgur](https://i.imgur.com/lqkeLxF.png)


## Model and Computational Complexity

모델 복잡도와 성능은 trade-off 관계임. reduction ratio $\gamma$를 모든 실험에서 16으로 셋팅하였음. 

실험 모델은 ResNet-50, SE-ResNet-50을 썼고, SE-ResNet-50이 ResNet-101에 근접하는 매우좋은 성능을 보여줌. 

- ResNet-50 : ~3.86 GFLOPS for 224 x 224 input image
- SE-ResNet-50 : ~3.87 GFLOPS -> 상대적으로 0.26%만 증가!

이는 squeeze, excitation, channel-wise scaling 전부 계산복잡도가 높지 않기 때문임

타이탄 X 썼을 때, 256 mini batch size 기준 ResNet-50은 190ms, SE-ResNet-50은 206ms가 걸렸음

$$\frac { 2 }{ r } \sum _{ s=1 }^{ S }{ { N }_{ s }\cdot { C }_{ s }^{ 2 } } $$

위 식은 SE block을 추가했을 때 추가되는 파라미터의 수임.

- $S$ : number of stages
- ${C}_{s}$ : dimension of the output channels
- ${N}_{s}$ : repepated block number for stage s

SE-ResNet-50이 ResNet-50 대비 ~2.5 million 개의 추가적인 파라미터가 들어감(ResNet-50만 따지면 25 million).  
마지막 단의 SE block은 사실 지워도 큰 차이가 없었음

## Implementation

- ImageNet data
- Random-size cropping to 224 x 224
- Random horizontal lflipping
- Normalize input image through mean channel subtraction
- Data balancing for mini batch sampling
- Momentum 0.9, batch size 1024
- Learning rate 0.6 -> x10 every 30 epochs
- 100 epochs
- Testing : center crop evaluation

## Experiments

### ImageNet classification

![Imgur](https://i.imgur.com/BWo3s95.png)

![Imgur](https://i.imgur.com/MiibvZx.png)

파라미터 증가량은 매우 적은데도 불구하고 SE block을 적용하면 굉장히 성능이 좋아지는 것을 확인할 수 있음.  
SE-ResNet-50이 ResNet-50에 비교해서 0.86% 좋아짐

![Imgur](https://i.imgur.com/a88ocU9.png)

망이 깊은 구조에 사용해도 성능 증가 효과가 상당함. 이는 deep한 구조의 모델에서 베이스로 넣기 충분희 유의미한 결과임

![Imgur](https://i.imgur.com/3ywuaXK.png)

SOTA들과 비교해서도 TOP 성능을 보여줌. (2017 ILSVRC 우승)

![Imgur](https://i.imgur.com/VQun2eY.png)

Scene classification, object detection 모두 좋은 성능을 보임

### Analysis and Interpreation

#### Reduction ratio

Reduction ratio $r$은 계산복잡도와 모델 용량을 결정하는 매우 중요한 하이퍼파라미터. Table 7을 보면 값의 변화에 따른 에러 변화량이 나와있음. 확실히 커지면 에러가 커지고 반면에 파라미터 수는 줄어듦. 

![Imgur](https://i.imgur.com/HysviWp.png)

#### The role of Excitation

![Imgur](https://i.imgur.com/TxLUc34.png)


각 클래스별 분포가 블럭이 바뀜에따라  어떻게 되는지 측정해봄.
Average activations for fifty uniformly sampled channels in the last SE block.

- 하위 부분일수록 거의 비슷한 분포를 따름. 이는 이는 초반 부분에 중요한 feature channel들이 공유가 된다고 할 수 있음
- 깊이가 깊어질수록 서로 달라짐. SE_4_6 이랑 SE 5_1 예시. 이는 SE block이 feature extraction과 feature specialization을 잘 하고있다는 것을 보여줌
- 네트워크 마지막 부분(SE_5_3 : classifier 직전)을 보면 클래스가 달라도 유사한 분포를 보여주고 있음. Recalibration을 제대로 못함. 파라미터 축소를 위해 제거하는게 나음


